In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from math import sqrt

In [236]:
! ls

action.csv    Cuisine.txt  nyc_inspection_data.zip	     violation.csv
Action.txt    mydb.db	   NY_City_Rest_SQL.ipynb	     Violation.txt
boroughs.csv  mydb.db;	   README.md			     webextract.csv
cuisine.csv   mydb.sqlite  RI_Webextract_BigApps_Latest.xls  WebExtract.txt


In [237]:
### Let's clean up violations

def pre_process(input_file, output_file):
    output = open(output_file, "w")
    with open(input_file, 'rb') as f:
        for line in f.readlines():
            line = line.split('","')
            #print line
            new = line
            # aggresively replace chars I don't need with ";"
            new = [x.replace(",", ";") for x in new]
            new = [x.replace("'", ";") for x in new]
            new = [x.replace('"', "") for x in new]
            # but I do want " at end as needed by sqlite
            new = [x.replace('\r\n', '"\r\n') for x in new]
            # last, prepend each entry with '"' as needed by sqlite
            new = '"' + str('","'.join(new))
            #print new
            output.write(str(new))
    f.close()
    output.close()
             

In [238]:
actions = pd.read_csv('Action.txt')

In [239]:
cuisines = pd.read_csv('Cuisine.txt')


In [240]:
pre_process('Violation.txt', 'violation.csv')



In [569]:
violations = pd.read_csv('violation.csv')


In [242]:
grades = pd.read_csv('WebExtract.txt')

In [243]:
actions.head()

,STARTDATE,ENDDATE,ACTIONCODE,ACTIONDESC
0,1901-01-01 00:00:00,2002-12-31 00:00:00,8,Establishment re-closed by DOHMH
1,2003-01-01 00:00:00,2010-07-18 00:00:00,8,Establishment re-closed by DOHMH
2,2010-07-19 00:00:00,2099-12-31 00:00:00,8,Establishment re-closed by DOHMH
3,1901-01-01 00:00:00,2002-12-31 00:00:00,A,NaN
4,2003-01-01 00:00:00,2010-07-18 00:00:00,A,NaN


In [451]:
cuisines.head()

,CUISINECODE,CODEDESC
0,2,African
1,3,American
2,5,Asian
3,15,Cajun
4,17,Caribbean


In [453]:
cuisines2 = cuisines.set_index(cuisines['CUISINECODE'])

In [459]:
cuisine_names = cuisines2['CODEDESC']

In [547]:
violations.head()

,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC
0,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,Current valid <a onmouseover=ShowContent(;P...
1,2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01A,Current valid <a onmouseover=ShowContent(;P...
2,2005-02-18 00:00:00,2007-06-30 00:00:00,Y,01A,Current valid <a onmouseover=ShowContent(;P...
3,2007-07-01 00:00:00,2008-06-30 00:00:00,Y,01A,Current valid permit; registration or other au...
4,2008-07-01 00:00:00,2009-08-01 00:00:00,Y,01A,Current valid permit; registration or other au...


In [598]:
import datetime

cutoff = pd.to_datetime('2014-01-01')

violations['ENDDATE'] = pd.to_datetime(violations['ENDDATE']) 
new_violations = violations[violations['ENDDATE'] > cutoff]
print violations['ENDDATE'].head() 
print cutoff
print new_violations['ENDDATE'].count() 
list_valid_violations = new_violations['VIOLATIONCODE']
list_valid_violations = list(set(list_valid_violations))
len(list_valid_violations)

0   2003-03-23
1   2005-02-17
2   2007-06-30
3   2008-06-30
4   2009-08-01
Name: ENDDATE, dtype: datetime64[ns]
2014-01-01 00:00:00
112


112

In [606]:
grades.count()

CAMIS           531935
DBA             531935
BORO            531935
BUILDING        531286
STREET          531934
ZIPCODE         531933
PHONE           531874
CUISINECODE     531935
INSPDATE        531935
ACTION          531066
VIOLCODE        520720
SCORE           498411
CURRENTGRADE    234325
GRADEDATE       232801
RECORDDATE      531935
dtype: int64

In [604]:
new_grades = grades.merge(new_violations,  left_on = 'VIOLCODE', right_on = 'VIOLATIONCODE')

In [607]:
new_grades.count()

CAMIS            520714
DBA              520714
BORO             520714
BUILDING         520079
STREET           520714
ZIPCODE          520714
PHONE            520670
CUISINECODE      520714
INSPDATE         520714
ACTION           520714
VIOLCODE         520714
SCORE            495636
CURRENTGRADE     232847
GRADEDATE        231362
RECORDDATE       520714
STARTDATE        520714
ENDDATE          520714
CRITICALFLAG     520714
VIOLATIONCODE    520714
VIOLATIONDESC    520714
dtype: int64

In [337]:
boroughs = pd.DataFrame({
    'boroughs': ['MANHATTAN', 'THE BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']
}, index=[1,2,3,4,5])


In [338]:
boroughs.head()

,boroughs
1,MANHATTAN
2,THE BRONX
3,BROOKLYN
4,QUEENS
5,STATEN ISLAND


In [343]:
boroughs_list = boroughs.groupby('boroughs').head(1)

In [357]:
### Let's get all restaurants

df_group_camis = grades.groupby('CAMIS')


In [356]:
df_group_camis.head(1).count()

CAMIS           25232
DBA             25232
BORO            25232
BUILDING        25171
STREET          25231
ZIPCODE         25230
PHONE           25214
CUISINECODE     25232
INSPDATE        25232
ACTION          24363
VIOLCODE        24032
SCORE           24326
CURRENTGRADE    24292
GRADEDATE       23865
RECORDDATE      25232
dtype: int64

In [291]:
df_rest_by_zipcode = df_group_camis[['CAMIS', 'ZIPCODE']].head(1).dropna().groupby('ZIPCODE').count()

In [294]:
df_rest_by_zipcode.head()

,CAMIS
ZIPCODE,
7005,1
10000,1
10001,509
10002,474
10003,675


In [298]:
df_zip_valid = df_rest_by_zipcode[ df_rest_by_zipcode['CAMIS'] > 100 ]

In [299]:
df_last_ins_on_camis = df_group_camis.apply(lambda g: g.sort('INSPDATE', ascending = False).head(1))
df_last_ins_on_camis.head()

,,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
CAMIS,,,,,,,,,,,,,,,,
30075445,0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
30112340,1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
30191841,2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
40356018,6,40356018,RIVIERA CATERER,3,2780,STILLWELL AVENUE,11224,7183723031,3,2014-06-10 00:00:00,D,10F,5,A,2014-06-10 00:00:00,2014-09-04 06:01:28.403000000
40356068,56182,40356068,TOV KOSHER KITCHEN,4,97-22,63 ROAD,11374,7188967788,50,2014-01-29 00:00:00,P,02B,12,NaN,NaN,2014-09-04 06:01:28.403000000


In [386]:
df_score_by_zipcode = df_last_ins_on_camis.groupby('ZIPCODE')
df_score_by_zipcode.head()

,,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
CAMIS,,,,,,,,,,,,,,,,
30075445,0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
30112340,1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
30191841,2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
40356018,6,40356018,RIVIERA CATERER,3,2780,STILLWELL AVENUE,11224,7183723031,3,2014-06-10 00:00:00,D,10F,5,A,2014-06-10 00:00:00,2014-09-04 06:01:28.403000000
40356068,56182,40356068,TOV KOSHER KITCHEN,4,97-22,63 ROAD,11374,7188967788,50,2014-01-29 00:00:00,P,02B,12,NaN,NaN,2014-09-04 06:01:28.403000000
40356151,19,40356151,BRUNOS ON THE BOULEVARD,4,88-25,ASTORIA BOULEVARD,11369,7183350505,3,2014-05-02 00:00:00,F,06D,10,A,2014-05-02 00:00:00,2014-09-04 06:01:28.403000000
40356442,15,40356442,KOSHER ISLAND,5,2206,VICTORY BOULEVARD,10314,7186985800,50,2014-05-20 00:00:00,D,10H,12,A,2014-05-20 00:00:00,2014-09-04 06:01:28.403000000
40356483,3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,08C,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
40356649,11,40356649,REGINA CATERERS,3,6409,11 AVENUE,11219,7182560829,3,2014-07-18 00:00:00,D,06A,12,A,2014-07-18 00:00:00,2014-09-04 06:01:28.403000000


In [387]:
means = (df_score_by_zipcode['SCORE'].mean())
counts = (df_score_by_zipcode['SCORE'].count())
sdevs = (df_score_by_zipcode['SCORE'].std())
locations = df_score_by_zipcode.count()['CAMIS']

In [388]:
means[10001]

9.1466942148760335

In [389]:
zips = df_zip_valid['CAMIS']
print len(zips)
print zips.keys()

92
Float64Index([10001.0, 10002.0, 10003.0, 10004.0, 10007.0, 10009.0, 10010.0,
              10011.0, 10012.0, 10013.0, 10014.0, 10016.0, 10017.0, 10018.0,
              10019.0, 10021.0, 10022.0, 10023.0, 10024.0, 10025.0, 10027.0,
              10028.0, 10029.0, 10032.0, 10033.0, 10034.0, 10036.0, 10038.0,
              10065.0, 10128.0, 10301.0, 10306.0, 10314.0, 10451.0, 10452.0,
              10458.0, 10461.0, 10462.0, 10463.0, 10467.0, 10468.0, 10472.0,
              11101.0, 11103.0, 11104.0, 11105.0, 11106.0, 11201.0, 11203.0,
              11204.0, 11205.0, 11206.0, 11207.0, 11208.0, 11209.0, 11211.0,
              11214.0, 11215.0, 11216.0, 11217.0, 11218.0, 11219.0, 11220.0,
              11221.0, 11222.0, 11223.0, 11225.0, 11226.0, 11229.0, 11230.0,
              11231.0, 11232.0, 11234.0, 11235.0, 11236.0, 11237.0, 11238.0,
              11249.0, 11354.0, 11355.0, 11358.0, 11361.0, 11368.0, 11369.0,
              11372.0, 11373.0, 11375.0, 11377.0, 11385.0, 11430.0, 11432

In [393]:
data_file = open('nyc_restaurants.csv', 'w')
scores = []
data_file.write('zipcode'+','+'mean score'+','+'standard error'+','+'number of locations' )
data_file.write('\n')

for z in zips.keys():
    z = int(z)
    m = means[z]
    c= counts[z]
    s = sdevs[z]/sqrt(c)
    l = locations[z]
    scores.append((str(z),m,s,l ) )
    data_file.write(str(z)+','+str(m)+','+str(s)+','+str(l) )
    data_file.write('\n')

In [394]:
len(scores)

92

In [328]:
!  cat nyc_restaurants.csv 

zipcode,mean score,standard error,number of locations
10001,9.14669421488,0.308901697428,484
10002,11.8686635945,0.402202259535,434
10003,11.6703821656,0.320951565545,628
10004,10.3821138211,0.526711565595,123
10007,10.6916666667,0.500251454044,120
10009,11.5909090909,0.397778402518,286
10010,11.327510917,0.517680147414,229
10011,11.4953271028,0.365144805111,428
10012,11.5602094241,0.359731477751,382
10013,12.0385487528,0.343583470906,441
10014,11.7023498695,0.40975010735,383
10016,11.7335025381,0.418528096392,394
10017,11.3661971831,0.422551921369,355
10018,11.7275541796,0.429500133716,323
10019,10.6301587302,0.278387773894,630
10021,12.5416666667,0.629104625756,144
10022,10.9421841542,0.339339388808,467
10023,10.7322404372,0.559547254313,183
10024,11.3,0.49079241333,190
10025,11.7909090909,0.575123965885,220
10027,11.3246753247,0.59121566085,154
10028,11.66,0.557607956512,200
10029,12.0215053763,0.706878932766,186
10032,12.7522123894,1.01315080483,113
10033,12.972972973,0.72381502013

In [395]:
sorted_scores = sorted(scores, key=lambda x: x[1])

In [396]:
sorted_scores

[('10451', 9.0723684210526319, 0.54327393132275426, 157),
 ('10001', 9.1466942148760335, 0.30890169742820034, 509),
 ('11430', 9.5, 0.71046402425395883, 148),
 ('11236', 9.865384615384615, 0.73913055875159872, 111),
 ('10452', 9.9222222222222225, 0.59444531951818169, 101),
 ('10472', 10.115384615384615, 0.63963378891832068, 108),
 ('11234', 10.127659574468085, 0.51530290949491031, 152),
 ('11217', 10.297071129707113, 0.43395628935426667, 254),
 ('10004', 10.382113821138212, 0.52671156559458721, 142),
 ('10306', 10.403846153846153, 0.46347536693149466, 111),
 ('11368', 10.460144927536232, 0.45484598436046375, 290),
 ('11369', 10.536082474226804, 0.70483447394971932, 104),
 ('10462', 10.537931034482758, 0.4993310361708409, 148),
 ('11201', 10.551401869158878, 0.39421688150155954, 345),
 ('11237', 10.584795321637428, 0.53618691155448228, 179),
 ('11231', 10.593103448275862, 0.52727308966482211, 159),
 ('11209', 10.620833333333334, 0.54325610327643292, 264),
 ('10019', 10.63015873015873, 0

In [358]:
df_score_by_boro = df_last_ins_on_camis.groupby('BORO')
df_score_by_boro.count()

,CAMIS,DBA,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
BORO,,,,,,,,,,,,,,
0,12,12,11,11,11,12,12,12,11,11,11,11,10,12
1,10201,10201,10188,10201,10201,10197,10201,10201,9891,9525,9560,9017,8876,10201
2,2365,2365,2365,2365,2364,2361,2365,2365,2281,2217,2216,2080,2040,2365
3,6072,6072,6070,6072,6072,6067,6072,6072,5850,5642,5620,5268,5129,6072
4,5625,5625,5580,5625,5625,5621,5625,5625,5418,5276,5272,4964,4845,5625
5,957,957,957,957,957,956,957,957,912,876,871,816,797,957


In [378]:
means = (df_score_by_boro['SCORE'].mean())
counts = (df_score_by_boro['SCORE'].count())
sdevs = (df_score_by_boro['SCORE'].std())
locations = df_score_by_boro.count()['CAMIS']

In [361]:
counts[0]

11

In [379]:
locations

BORO
0       12
1    10201
2     2365
3     6072
4     5625
5      957
Name: CAMIS, dtype: int64

In [371]:
boroughs_list['boroughs'][1]

'MANHATTAN'

In [382]:
scores = []

for b in range(1,len(boroughs_list['boroughs'])+1):
    boro = boroughs_list['boroughs'][b]
    m = means[b]
    c= counts[b]
    s = sdevs[b]/sqrt(c)
    l = locations[b]
    scores.append((boro,m,s,l) )

In [383]:
scores

[('MANHATTAN', 11.336715481171549, 0.079554979655562685, 10201),
 ('THE BRONX', 10.735559566787003, 0.1520699762143875, 2365),
 ('BROOKLYN', 11.467615658362989, 0.11020734729093988, 6072),
 ('QUEENS', 11.744119878603945, 0.11123141307427269, 5625),
 ('STATEN ISLAND', 11.732491389207807, 0.24953516703633261, 957)]

In [407]:
df_cuisine_vs_violation = pd.crosstab(grades.VIOLCODE,grades.CUISINECODE)

In [409]:
df_cuisine_valid = df_cuisine_vs_violation.sum()

In [412]:
df_cuisine_valid[25]

49

In [418]:
df_cuisine_valid

CUISINECODE
0        131
1        300
2       1843
3     125458
4        632
5       7654
6        248
7       3657
8      16100
9        803
10       799
11        37
12      1021
13       670
14     14860
15       179
16         2
17     16100
18      7442
19         4
20     60880
21       438
22      1303
23       983
24       679
25        49
26       127
27      9590
28      1829
29      6303
       ...  
54      4169
55     18419
56      3313
57       284
58        59
59       811
60       410
61      1886
62     25687
63     10645
64       722
66       269
67      2023
68       552
69      6510
70      3057
71       149
72      3245
73      1025
74        46
75       576
76       203
77     13090
78      1797
80       614
81      2836
82      6843
83      1533
84      1958
99      1881
dtype: int64

In [508]:
df_score_by_cuisine = grades.groupby('CUISINECODE')
df_score_by_cuisine.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,08C,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,02G,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
5,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10F,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
6,40356018,RIVIERA CATERER,3,2780,STILLWELL AVENUE,11224,7183723031,3,2014-06-10 00:00:00,D,10F,5,A,2014-06-10 00:00:00,2014-09-04 06:01:28.403000000
7,40356068,TOV KOSHER KITCHEN,4,97-22,63 ROAD,11374,7188967788,50,2013-01-17 00:00:00,D,06D,13,Z,2013-01-17 00:00:00,2014-09-04 06:01:28.403000000
8,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,04M,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
9,40356731,TASTE THE TROPICS ICE CREAM,3,1839,NOSTRAND AVENUE ...,11226,7188560821,43,2014-07-14 00:00:00,D,10F,12,A,2014-07-14 00:00:00,2014-09-04 06:01:28.403000000


In [509]:
means = (df_score_by_cuisine['SCORE'].mean())
counts = (df_score_by_cuisine['SCORE'].count())
sdevs = (df_score_by_cuisine['SCORE'].std())

In [523]:
sdevs[0]

19.725651643326643

In [511]:
list_cuisinecodes = df_cuisine_valid.keys()

In [512]:
list_cuisinecodes

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68,
            69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 99],
           dtype='int64', name=u'CUISINECODE')

In [513]:
int(df_cuisine_valid[99])

1881

In [514]:
df_cuisine_valid

CUISINECODE
0        131
1        300
2       1843
3     125458
4        632
5       7654
6        248
7       3657
8      16100
9        803
10       799
11        37
12      1021
13       670
14     14860
15       179
16         2
17     16100
18      7442
19         4
20     60880
21       438
22      1303
23       983
24       679
25        49
26       127
27      9590
28      1829
29      6303
       ...  
54      4169
55     18419
56      3313
57       284
58        59
59       811
60       410
61      1886
62     25687
63     10645
64       722
66       269
67      2023
68       552
69      6510
70      3057
71       149
72      3245
73      1025
74        46
75       576
76       203
77     13090
78      1797
80       614
81      2836
82      6843
83      1533
84      1958
99      1881
dtype: int64

In [515]:
food = dict(cuisine_names)

In [516]:
new_food = {}
for f in food.keys():
    new_food[f]= str(food[f]).decode("ISO-8859-1").encode("utf-8")

    
##.decode("ISO-8859-1").encode("utf-8"))

In [520]:
new_food

{0: 'Not Listed/Not Applicable',
 1: 'Afghan',
 2: 'African',
 3: 'American ',
 4: 'Armenian',
 5: 'Asian',
 6: 'Australian',
 7: 'Bagels/Pretzels',
 8: 'Bakery',
 9: 'Bangladeshi',
 10: 'Barbecue',
 11: 'Basque',
 12: 'Bottled beverages, including water, sodas, juices, etc.',
 13: 'Brazilian',
 14: 'Caf\xc3\xa9/Coffee/Tea',
 15: 'Cajun',
 16: 'Californian',
 17: 'Caribbean',
 18: 'Chicken',
 19: 'Chilean',
 20: 'Chinese',
 21: 'Chinese/Cuban',
 22: 'Chinese/Japanese',
 23: 'Continental',
 24: 'Creole',
 25: 'Creole/Cajun',
 26: 'Czech',
 27: 'Delicatessen',
 28: 'Vietnamese/Cambodian/Malaysia',
 29: 'Donuts',
 30: 'Eastern European',
 31: 'Egyptian',
 32: 'English',
 33: 'Ethiopian',
 34: 'Filipino',
 35: 'French',
 36: 'Fruits/Vegetables',
 37: 'German',
 38: 'Greek',
 39: 'Hamburgers',
 40: 'Hawaiian',
 41: 'Hotdogs',
 42: 'Hotdogs/Pretzels',
 43: 'Ice Cream, Gelato, Yogurt, Ices',
 44: 'Indian',
 45: 'Indonesian',
 46: 'Iranian',
 47: 'Irish',
 48: 'Italian',
 49: 'Japanese',
 50: 

In [502]:
new_num_v ={}
for f in num_violations.keys():
    new_num_v[f]= str(num_violations[f]).decode("ISO-8859-1").encode("utf-8")


In [503]:
new_num_v

{0: '131',
 1: '300',
 2: '1843',
 3: '125458',
 4: '632',
 5: '7654',
 6: '248',
 7: '3657',
 8: '16100',
 9: '803',
 10: '799',
 11: '37',
 12: '1021',
 13: '670',
 14: '14860',
 15: '179',
 16: '2',
 17: '16100',
 18: '7442',
 19: '4',
 20: '60880',
 21: '438',
 22: '1303',
 23: '983',
 24: '679',
 25: '49',
 26: '127',
 27: '9590',
 28: '1829',
 29: '6303',
 30: '1507',
 31: '348',
 32: '334',
 33: '297',
 34: '567',
 35: '7409',
 36: '51',
 37: '855',
 38: '2555',
 39: '7663',
 40: '68',
 41: '433',
 42: '145',
 43: '3670',
 44: '8170',
 45: '213',
 46: '79',
 47: '4422',
 48: '25637',
 49: '18374',
 50: '7532',
 51: '2638',
 52: '6957',
 53: '26063',
 54: '4169',
 55: '18419',
 56: '3313',
 57: '284',
 58: '59',
 59: '811',
 60: '410',
 61: '1886',
 62: '25687',
 63: '10645',
 64: '722',
 66: '269',
 67: '2023',
 68: '552',
 69: '6510',
 70: '3057',
 71: '149',
 72: '3245',
 73: '1025',
 74: '46',
 75: '576',
 76: '203',
 77: '13090',
 78: '1797',
 80: '614',
 81: '2836',
 82: '6

In [475]:
num_violations = dict(df_cuisine_valid)

In [476]:
num_violations

{0: 131,
 1: 300,
 2: 1843,
 3: 125458,
 4: 632,
 5: 7654,
 6: 248,
 7: 3657,
 8: 16100,
 9: 803,
 10: 799,
 11: 37,
 12: 1021,
 13: 670,
 14: 14860,
 15: 179,
 16: 2,
 17: 16100,
 18: 7442,
 19: 4,
 20: 60880,
 21: 438,
 22: 1303,
 23: 983,
 24: 679,
 25: 49,
 26: 127,
 27: 9590,
 28: 1829,
 29: 6303,
 30: 1507,
 31: 348,
 32: 334,
 33: 297,
 34: 567,
 35: 7409,
 36: 51,
 37: 855,
 38: 2555,
 39: 7663,
 40: 68,
 41: 433,
 42: 145,
 43: 3670,
 44: 8170,
 45: 213,
 46: 79,
 47: 4422,
 48: 25637,
 49: 18374,
 50: 7532,
 51: 2638,
 52: 6957,
 53: 26063,
 54: 4169,
 55: 18419,
 56: 3313,
 57: 284,
 58: 59,
 59: 811,
 60: 410,
 61: 1886,
 62: 25687,
 63: 10645,
 64: 722,
 66: 269,
 67: 2023,
 68: 552,
 69: 6510,
 70: 3057,
 71: 149,
 72: 3245,
 73: 1025,
 74: 46,
 75: 576,
 76: 203,
 77: 13090,
 78: 1797,
 80: 614,
 81: 2836,
 82: 6843,
 83: 1533,
 84: 1958,
 99: 1881}

In [524]:
scores = []

for code in list_cuisinecodes:
    m = means[code]
    c= counts[code]
    s = sdevs[code]/sqrt(c)
    v = int(new_num_v[code])
    f = new_food[code]
    
    if v > 100:
        scores.append((f,m,s,v))

In [525]:
scores

[('Not Listed/Not Applicable', 22.262711864406779, 1.8158934224350858, 131),
 ('Afghan', 22.079710144927535, 0.83311127246986538, 300),
 ('African', 26.658836689038033, 0.45486540807469628, 1843),
 ('American ', 21.354490696464826, 0.041713159443338095, 125458),
 ('Armenian', 17.233449477351915, 0.40865664646012262, 632),
 ('Asian', 26.057228505245945, 0.21557599197048327, 7654),
 ('Australian', 22.257510729613735, 1.1245882063845891, 248),
 ('Bagels/Pretzels', 21.06962921030286, 0.23343476916521777, 3657),
 ('Bakery', 23.06542899503642, 0.12982566974763338, 16100),
 ('Bangladeshi', 26.586649550706035, 0.65418980624005008, 803),
 ('Barbecue', 20.859354838709677, 0.45447891836054854, 799),
 ('Bottled beverages, including water, sodas, juices, etc.',
  17.851214574898787,
  0.40427666581749311,
  1021),
 ('Brazilian', 22.822503961965136, 0.51021837909325785, 670),
 ('Caf\xc3\xa9/Coffee/Tea', 17.125518307681496, 0.1034563357741395, 14860),
 ('Cajun', 17.214285714285715, 0.5886086483756457

In [486]:
'Caf\xe9/Coffee/Tea'.decode("ISO-8859-1").encode("utf-8")

'Caf\xc3\xa9/Coffee/Tea'

In [479]:
len(scores)

75

In [526]:
sorted_scores = sorted(scores, key=lambda x: x[1])

In [527]:
sorted_scores

[('Hotdogs/Pretzels', 14.51923076923077, 1.0003916170255032, 145),
 ('Soups & Sandwiches', 14.961199294532628, 0.40940736861991633, 576),
 ('Ethiopian', 15.346153846153847, 0.42520183303084802, 297),
 ('Donuts', 15.536336731338375, 0.14352764563010978, 6303),
 ('Hotdogs', 15.5413870246085, 0.4393868291224805, 433),
 ('Ice Cream, Gelato, Yogurt, Ices',
  15.943380040034315,
  0.19522045142522071,
  3670),
 ('Sandwiches', 16.438442211055275, 0.14547803792438158, 6510),
 ('Juice, Smoothies, Fruit Salads',
  16.8989898989899,
  0.23832715850164446,
  2638),
 ('Caf\xc3\xa9/Coffee/Tea', 17.125518307681496, 0.1034563357741395, 14860),
 ('Sandwiches/Salads/Mixed Buffet',
  17.19119623655914,
  0.22072423684354653,
  3057),
 ('Cajun', 17.214285714285715, 0.58860864837564575, 179),
 ('Armenian', 17.233449477351915, 0.40865664646012262, 632),
 ('Other', 17.392756083757781, 0.36502101968711737, 1881),
 ('Hamburgers', 17.512352309344791, 0.1324045098073057, 7663),
 ('Bottled beverages, including wa

In [671]:
df_cuisines_vs_violations =pd.crosstab(new_grades.VIOLCODE,new_grades.CUISINECODE)

In [673]:
cui_count_dic = df_cuisines_vs_violations.sum()
cui_count_dic

CUISINECODE
0        131
1        300
2       1843
3     125453
4        632
5       7654
6        248
7       3657
8      16100
9        803
10       799
11        37
12      1021
13       670
14     14860
15       179
16         2
17     16100
18      7442
19         4
20     60880
21       438
22      1303
23       983
24       679
25        49
26       127
27      9590
28      1829
29      6303
       ...  
54      4169
55     18419
56      3313
57       284
58        59
59       811
60       410
61      1886
62     25686
63     10645
64       722
66       269
67      2023
68       552
69      6510
70      3057
71       149
72      3245
73      1025
74        46
75       576
76       203
77     13090
78      1797
80       614
81      2836
82      6843
83      1533
84      1958
99      1881
dtype: int64

In [661]:
df_violations_vs_cuisines =pd.crosstab(new_grades.CUISINECODE,new_grades.VIOLCODE)

In [714]:
df_violations_vs_cuisines['04D'][21]

0

In [664]:
df_violations_vs_cuisines

VIOLCODE,02A,02B,02C,02D,02E,02F,02G,02H,02I,03A,...,20A,20B,20D,20E,20F,22A,22B,22C,22E,99B
CUISINECODE,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,3,0,0,0,...,0,0,4,0,2,1,0,3,0,0
1,0,4,0,0,0,0,12,1,0,0,...,0,0,0,1,2,0,0,3,0,5
2,1,142,5,0,0,0,113,22,0,3,...,3,0,8,1,12,0,3,19,0,4
3,130,3836,110,18,6,2,10890,673,10,87,...,242,2,469,139,1065,633,120,1084,24,536
4,0,6,0,0,0,0,42,4,0,1,...,5,0,6,0,4,5,2,8,0,2
5,4,426,12,0,0,0,731,45,0,11,...,12,0,26,6,53,25,5,61,0,30
6,1,5,0,0,0,0,21,0,0,1,...,0,0,2,0,0,0,0,5,0,0
7,0,80,3,0,0,0,428,12,0,4,...,13,0,7,1,20,10,1,30,0,16
8,6,954,8,0,0,0,1264,44,1,35,...,41,0,47,9,85,18,9,202,0,82


In [659]:
vio_count_dic = df_violations_vs_cuisines.sum()
vio_count_dic

VIOLCODE
02A      498
02B    24523
02C      550
02D       52
02E       19
02F       10
02G    45819
02H     3160
02I       42
03A      593
03B      331
03C      474
03D       90
03E       22
03G       20
04A     9264
04B       38
04C     4726
04D      496
04E      324
04F       89
04G       16
04H    15954
04I       25
04J     6272
04K     3306
04L    38306
04M    14568
04N    21508
04O      541
       ...  
10J     2472
15E       11
15H        1
15I      453
15J      464
15K      391
15L     3128
15S      298
15T       90
16A      916
16B     5852
16C      188
16E       16
16F       10
18A       67
18B       29
18C       32
18D      137
18F       69
18G       38
20A      884
20B       18
20D     1945
20E      476
20F     3775
22A     1746
22B      396
22C     4401
22E       61
99B     2257
dtype: int64

In [643]:
df_violation_desc = new_violations.groupby('VIOLATIONCODE').head(1)

In [644]:
df_violation_desc.index = df_violation_desc['VIOLATIONCODE']

In [646]:
df_violation_desc= df_violation_desc['VIOLATIONDESC']

In [648]:
df_violation_desc['02A']

'Food not cooked to required minimum temperature.'

In [651]:
df_violation_desc.keys()

Index([u'02A', u'02B', u'02C', u'02D', u'02E', u'02F', u'02G', u'02H', u'02I',
       u'02J', 
       ...
       u'20B', u'20C', u'20D', u'20E', u'20F', u'22A', u'22B', u'22C', u'22E',
       u'99B'],
      dtype='object', name=u'VIOLATIONCODE', length=112)

In [706]:
vio_dic ={}
for k in df_violations_vs_cuisines.sum().keys():
    if k in df_violation_desc.keys():
        vio_dic[k]= str(df_violation_desc[k]).decode("ISO-8859-1").encode("utf-8")


In [712]:
vio_dic

{'02A': 'Food not cooked to required minimum temperature.',
 '02B': 'Hot food item not held at or above 140\xc2\xba F.',
 '02C': 'Hot food item that has been cooked and refrigerated is being held for service without first being reheated to 1 65\xc2\xba F or above within 2 hours.',
 '02D': 'Precooked potentially hazardous food from commercial food processing establishment that is supposed to be heated; but is not heated to 140\xc2\xba F within 2 hours.',
 '02E': 'Whole frozen poultry or poultry breasts; other than a single portion; is being cooked frozen or partially thawed.',
 '02F': 'Meat; fish or molluscan shellfish served raw or undercooked without prior notification to customer.',
 '02G': 'Cold food item held above 41\xc2\xba F (smoked fish and reduced oxygen packaged foods above 38 \xc2\xbaF) except during necessary preparation.',
 '02H': 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140\xc2\xba F to 70\xc2\xba F or less within 2 h

In [708]:
cui_dic ={}
for k in df_cuisines_vs_violations.sum().keys():
    cui_dic[k]= str(new_food[k]).decode("ISO-8859-1").encode("utf-8")

In [709]:
cui_dic

{0: 'Not Listed/Not Applicable',
 1: 'Afghan',
 2: 'African',
 3: 'American ',
 4: 'Armenian',
 5: 'Asian',
 6: 'Australian',
 7: 'Bagels/Pretzels',
 8: 'Bakery',
 9: 'Bangladeshi',
 10: 'Barbecue',
 11: 'Basque',
 12: 'Bottled beverages, including water, sodas, juices, etc.',
 13: 'Brazilian',
 14: 'Caf\xc3\x83\xc2\xa9/Coffee/Tea',
 15: 'Cajun',
 16: 'Californian',
 17: 'Caribbean',
 18: 'Chicken',
 19: 'Chilean',
 20: 'Chinese',
 21: 'Chinese/Cuban',
 22: 'Chinese/Japanese',
 23: 'Continental',
 24: 'Creole',
 25: 'Creole/Cajun',
 26: 'Czech',
 27: 'Delicatessen',
 28: 'Vietnamese/Cambodian/Malaysia',
 29: 'Donuts',
 30: 'Eastern European',
 31: 'Egyptian',
 32: 'English',
 33: 'Ethiopian',
 34: 'Filipino',
 35: 'French',
 36: 'Fruits/Vegetables',
 37: 'German',
 38: 'Greek',
 39: 'Hamburgers',
 40: 'Hawaiian',
 41: 'Hotdogs',
 42: 'Hotdogs/Pretzels',
 43: 'Ice Cream, Gelato, Yogurt, Ices',
 44: 'Indian',
 45: 'Indonesian',
 46: 'Iranian',
 47: 'Irish',
 48: 'Italian',
 49: 'Japanese

In [738]:
ratio_results = []
sum_vio = sum(vio_count_dic)*1.0
sum_cui = sum(cui_count_dic)*1.0
assert sum_vio == sum_cui

for vio in vio_dic:
    for cui in cui_dic:
        count = int(str(df_violations_vs_cuisines[vio][cui]).decode("ISO-8859-1").encode("utf-8"))
        vio_count = int(str(vio_count_dic[vio]).decode("ISO-8859-1").encode("utf-8"))
        cui_count = int(str(cui_count_dic[cui]).decode("ISO-8859-1").encode("utf-8"))
        cui_name = cui_dic[cui]
        vio_name = vio_dic[vio]
        ratio = ((count*1.0/ cui_count)/(vio_count/sum_vio))
        if count > 100:
            ratio_results.append(((cui_name,vio_name), ratio, count)) 
            




In [739]:
ratio_results

[(('Chinese',
   'Food worker does not wash hands thoroughly after using the toilet; coughing; sneezing; smoking; eating; preparing raw foods or otherwise contaminating hands.'),
  2.086547637340511,
  121),
 (('African',
   'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed; raised; spaced or movable to allow accessibility for cleaning on all sides; above and underneath the unit.'),
  0.83799470411657151,
  197),
 (('American ',
   'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed; raised; spaced or movable to allow accessibility for cleaning on all sides; above and underneath the unit.'),
  1.0349803685127723,
  16562),
 (('Asian',
   'Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly

In [740]:
sorted_ratio_results=sorted(ratio_results, key=lambda x: x[1], reverse=True)

In [742]:
sorted_ratio_results[:20]

[(('Japanese',
   'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.'),
  3.2441362822892623,
  541),
 (('Caf\xc3\x83\xc2\xa9/Coffee/Tea',
   '\xc2\x93Choking first aid\xc2\x94 poster not posted. \xc2\x93Alcohol and pregnancy\xc2\x94 warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric); latex gloves; sign not posted. Inspection report sign not posted.'),
  3.1528179028256873,
  175),
 (('Juice, Smoothies, Fruit Salads',
   'Food Protection Certificate not held by supervisor of food operations.'),
  3.0895406873894369,
  145),
 (('Donuts',
   'Accurate thermometer not provided in refrigerated or hot holding equipment.'),
  3.0372675010032575,
  130),
 (('Ice Cream, Gelato, Yogurt, Ices',
   'Food Protection Certificate not held by supervisor of food operations.'),
  2.9559150772025431,
  193),
 (('Thai', 'Thawing procedures improper.'), 2.632963991516

In [743]:
sum_item = 0
for item in sorted_ratio_results[:20]:
    print item[1]
    sum_item+= item[1]
print sum_item/20

3.24413628229
3.15281790283
3.08954068739
3.037267501
2.9559150772
2.63296399152
2.36927766719
2.32605228091
2.25895790403
2.19707802972
2.10840246875
2.09466545479
2.09139964377
2.08654763734
2.07991629945
2.06487261145
1.98502112889
1.90085711385
1.89562161163
1.8836203887
2.37274658413


In [632]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [50]:
c.execute('''''')


# SQL
Structured Query Language (pronounced like "sequel")

**Key nouns:**
    - "Relational" database
    - Table
    - Row
    - Column
    - (Primary key)

**Key verbs:**
    - `CREATE TABLE` / `DROP TABLE` to modify the structure of the database
    - `SELECT` to read out some rows
    - `INSERT` / `UPDATE` / `DELETE` to modify (or delete) some rows of tables

**Key modifiers:**
    - `WHERE`: impose condition
    - `ORDER BY`: sort
    - `GROUP BY`: aggregate (e.g. pivot tables in Excel)
    - `JOIN`: combining tables in one query

## The relational data model

Here's a diagram of a simple database for employee work-planning in the "relational model":

<div>
 <div>
 <div>
 <table>
 
 <tr>
  <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;employee_id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </th> 
 <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;name&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;Phone&nbsp;&nbsp;&nbsp;  </th>
 </tr>
 
 <tr>
 <td>1</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Jenny Smith&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;867-5309&nbsp;&nbsp;&nbsp;  </td>
 </tr>

 <tr>
 <td>2</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;John Doe&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;555-1234&nbsp;&nbsp;&nbsp;  </td>
 </tr>

 <tr>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 </tr>

 <tr>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 </tr>
</table>
</div>
<div style="text-align: center"> <span> Table: <tt>Employees</tt> </span>
</div>
</div>

<br />

 <div>
 <div>
 <table>
 
 <tr>
 <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;task_id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </th> 
 <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;deadline&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;description&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 </tr>
 
 <tr>
 <td>1</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2014-07-18&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;Save the world.&nbsp;&nbsp;&nbsp;  </td>
 </tr>

 <tr>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 </tr>
</table>
</div>
<div style="text-align: center"> <span> Table: <tt>Tasks</tt> </span>
</div>
</div>

<br />

<div>
 <div>
 <table>
 
 <tr>
 <th> &nbsp;&nbsp;assignment_id&nbsp;&nbsp;&nbsp;&nbsp; </th> 
 <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;task_id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </th> 
 <th>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;employee_id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;start_date&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;end_date&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 <th> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </th>
 </tr>
 
 
 <tr>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
 </tr>
 
 <tr>
 <td>1234</td>
 <td>1</td>
 <td>2</td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2014-05-18&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2014-07-18&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 <td> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;...&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  </td>
 </tr>

</table>
</div>
<div style="text-align: center"> <span> Table: <tt>Assignments</tt> </span>
</div>
</div>



The database consists of:
 - **Tables**: Employees, Tasks, and Assignments.  
      > Each table represents a different kind of "relation" between bits of data.
    
 - Each of the tables has many **rows** (sometimes called "relations").  For instance, each row of the `Employees` table represents information about a single employee:  
 
      > Elements of a single column are _homogeneous_ (of the same kind, or data type) but apply to _different logical entities_.
      
 - The information about a single employee is spread amongst the **columns** of the table:
 
      > Elements of a single row are _heterogeneous_ but apply to _a single logical entity_.
      
 - It is good practice to be able to specify each row of each table by a **primary key**: that is by some column, or small combination of columns, that uniquely specifies the row.  The columns `employee_id`, `task_id`, and `assignment_id` are all of this type.  This way, if you want to refer to this row in another table you can do it succinctly by this primary key.
      
Here are some examples of types of "relations" (rows) that occur in practice:
   1. Storing one-to-one bits of data for some sort of object: i.e. for each employee we store one name and one phone number; for each project we store one deadline and one description
   2. Storing one-to-many or many-to-many bits of data: i.e. if we needed to be able to store _many_ phone numbers for each employee, we'd have a new table `Employee_Phone_Numbers` with columns `employee_id` and `phone_number` (the two together can be a _composite primary key_).
    
   Alternate, but **worse**, ways to do it would be: 
       - Adding columns to the `Employees` table for `phone_number_1` to `phone_number_10`.  This is bad because it makes it hard to query against the phone numbers and is inflexible if you want to change the number of phone numbers.  In the _object oriented_ or _key-value_ you could simply make phone_numbers a list or array, and all would be well -- this is not available in the relational model. 
       - Having the `Employee_Phone_Numbers` table duplicate the employee name and other information.  This is bad because duplication of information across tables wastes space and can lead to update errors.
   

We're going to start by doing all our examples in `sqlite`.   Then at the end we'll talk a little bit about setting up and using `postgres`.  They both speak (mostly) SQL, so at the end we'll talk about why you might use one of them (vs the other, and vs other options).

Both have command-line clients (`sqlite3` and `psql`) but there is also a convenient IPython %sql "magic".


## Schemas and creating databases
A __schema__ is a description of the structure of a database (tables, rows, etc.).  We proceed by example:

In [41]:
%%sql sqlite://
DROP TABLE IF EXISTS Employees;
CREATE TABLE Employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    title TEXT NOT NULL,
    phone_number TEXT NOT NULL
);

DROP TABLE IF EXISTS Tasks;
CREATE TABLE Tasks (
    task_id INTEGER PRIMARY KEY,
    deadline DATE NOT NULL,
    status INT NOT NULL DEFAULT 0,
    description TEXT
);


DROP TABLE IF EXISTS Assignments;
CREATE TABLE Assignments (
    assignment_id INTEGER PRIMARY KEY,
    task_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    charge_code TEXT NOT NULL,  -- This should probably be its own table!
    fraction_assignment REAL NOT NULL
);

DROP TABLE IF EXISTS Pension_Choices;
CREATE TABLE Pension_Choices (
    pension_id INTEGER PRIMARY KEY,
    employee_id INTEGER NOT NULL, 
    plan_name TEXT NOT NULL,
    funding_amount INTEGER NOT NULL);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [43]:
%%sql sqlite:// 
    
SELECT * FROM actions LIMIT 10

(sqlite3.OperationalError) no such table: actions [SQL: u'SELECT * FROM actions LIMIT 10']


## Inserting data
The basic way to create a new row is

    INSERT INTO table_name [(column_name_1, column_name_2, ...)] VALUES (value_1, value_2, ...);

If you don't give the optional list of column names, then it is assumed that the values are in the order that the fields appeared in the `CREATE TABLE` statement.  Giving the list of column names is a good idea: It doesn't break when you change your schema, it lets you take advantage of default values and auto-incrementing primary keys, etc.

**Gotchas:**
 - For strings you 'must' use single quotes.  (SQLite and MySQL don't care.  Postgres does.)
 - The primary key is required to uniquely specify a row.  If you try to insert something that violates this, you will get an error.

In [ ]:
%%sql
INSERT INTO Employees VALUES (1, 'Jenny Smith', '2010-12-1', 'Chief Music Office', '867-5309');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('CEO', 'John Doe', '555-1234', '2011-12-1');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 1', '555-1201', '2012-12-01');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 2', '555-1202', '2012-12-03');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 3', '555-1203', '2012-12-05');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 4', '555-1204', '2012-12-07');

INSERT INTO Tasks (deadline, description, status) VALUES ('2012-12-11', 'Hire worker bees.', 1);
INSERT INTO Tasks (deadline, description) VALUES ('2014-07-18', 'Save the world.');
INSERT INTO Tasks (deadline, description, status) VALUES ('2014-12-11', 'Cost cutting on worker bees.', 0);


INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (1, 2, '2012-01-01', '2012-12-31', 0.5, 'INTERNAL');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 3, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 4, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 5, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 6, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 2, '2013-01-01', '2014-07-18', 0.75, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (3, 2, '2013-01-01', '2014-07-18', 0.25, 'INTERNAL');

INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (3, 'Retirement Plan A', '3000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (4, 'Retirement Plan A', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (5, 'Retirement Plan B', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (6, 'Retirement Plan B', '1000');


Note that the primary key is required to be _unique_.  That means that if we were to try the following line
```SQL
INSERT INTO Employees VALUES (1, 'Jenny Smith', '2010-12-1', 'Chief Music Office', '867-5309');
```
_again_, it would fail.  (Even if the fields after the 1 were changed to be different.)

## Selecting data
The basic way to read rows from a single table is

```SQL
SELECT expression1, expression2, ... FROM table_name [WHERE condition] [GROUP BY columns] [ORDER BY columns] [LIMIT number];
```

Brackets denote parts which are optional.  Let us explain the parts:
   1. __"Expressions": __
   
   The basic example is just a column name.  Another example is the placeholder `*`: it indicates _all_ columns.  You can also perform arithmetic (e.g. `column_1 + column_2`) and similar minor operations.
   2. __`WHERE` ("condition"):__
   
   An example would be `column_name = value`, and for partial string matches there is a special operator `LIKE`.  You can combine these with the usual binary operators (AND, OR) and parentheses for grouping.
   3. __`GROUP BY` (aggregation):__
   
   Add a `GROUP BY column_name1, ...` term to aggregate the results along the listed columns.   This means that for _other_ columns you mention in the expressions you must explicitly specify an _aggregate method_ by writing e.g. `SUM(column_1)`.
   
   You can use these aggregate methods even without a GROUP BY, to get information for the whole table: e.g.
  ```SQL 
SELECT COUNT(1) FROM table_name;
SELECT SUM(colum) FROM TABLE_name;
   ```
   return the number of rows, and the sum of the named column, respectively.
   
   4. __`ORDER BY` (sorting):__
   
   You can add a `SORT BY column_name1, column_name2, ...` to sort the results by the given column (applied in left-to-right order).  By default the sorting is ascending.  If you want it to be descending write e.g. `column_name1 DESC` (you can also write `ASC` for ascending to disambiguate).
   
   5. __`LIMIT`:__
   
   If you want to just get at most the first N rows that would be returned, add on `LIMIT N`.
   
   6. __`AS` (named expressions):__
   
   You can specify new names for the expressions that you're selecting.  This is especially useful when you're using aggregate functions: 
```SQL
SELECT SUM(salary) as 'Total Salary' FROM ...
```
  
Finally two remarks: SQL is not case sensitive.  It is just common to use all-caps for the commands, and normal case for table / column names.  It is also whitespace and new-line insensitive, so you can format your commands on multiple lines as you please.


Now for a bunch of examples:

In [ ]:
%sql SELECT * FROM Employees

In [ ]:
%sql SELECT * FROM Tasks;

In [ ]:
%sql SELECT * FROM Assignments;

In [ ]:
%sql SELECT COUNT(1) as Count FROM Tasks WHERE status=0;

In [ ]:
%%sql
SELECT assignment_id, julianday(end_date) - julianday(start_date) 
    as duration FROM Assignments WHERE employee_id=2;

In [ ]:
%%sql
SELECT task_id, SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
    FROM Assignments
    GROUP BY task_id
    ORDER BY Total_Worker_Days DESC;

## Combining tables: Joins and subqueries

It's pretty boring to only use one table.  We can combine data from multiple tables in two main ways: joins and subqueries.

**Nicknames:**

There's a common bit of syntax involved in each: you can write

    table_name name

to say that table `table_name` is going to participate in this query with a "nickname" of name.  Then, to refer to one of its columns you write

    name.column_name

You could also have not given it the nickname, and then you could've written `table_name.column_name`... but that gets old fast.  Also, in some fancy examples the same table enters into a query twice -- at which point you need to name them.


**Subqueries:**
It turns out that both the expressions you were SELECTing __and__ the "table" you were selecting FROM can be subqueries (__and__ quantities in WHEREs)!  Let's just see it by example:

In [ ]:
%%sql
SELECT a.task_id, 
    (SELECT description 
     FROM Tasks t WHERE t.task_id=a.task_id) as description, 
    SUM((julianday(a.end_date) - julianday(a.start_date)) 
                * a.fraction_assignment) as Total_Worker_Days
    FROM Assignments a
    GROUP BY a.task_id
    ORDER BY Total_Worker_Days DESC;

In [ ]:
%%sql
SELECT ad.task_id, ad.Total_Worker_Days 
    FROM (SELECT task_id, SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
            FROM Assignments
            GROUP BY task_id
            ORDER BY Total_Worker_Days DESC) as ad
    WHERE ad.task_id=(SELECT task_id FROM Tasks WHERE description LIKE 'Save%');

**Joins:**
Many things that can be expressed by a subquery can be expressed more clearly (and will often run faster) as a `join`.   

The basic idea of joins is to imagine taking the _product_ of your two tables: i.e. a row is now an ordered pair of a row of table 1 and a row of table 2.  You could -- of course -- do this and make it its own table, but the ability to write JOINs means that you don't have to waste the storage space and risk un-needed duplication of data.

The join in the previous paragraph is called the full or outer join, because there are no conditions.  In practice, you'll want to take the subset of the product where some condition holds (i.e. "join" the two tables along a common id column).  This can be expressed pretty much exactly as I just said, in SQL:
    
```SQL
       SELECT * FROM table1, table2
                WHERE table1.column_name1 = table2.column_name2;
```

In SQL terms, this is called an `INNER JOIN`: You do not get a row in the result of the SELECT unless you can match a row in table 1 with a match in table 2 along the given columns.  There is alternate syntax for it:
```SQL
SELECT * FROM table1
        INNER JOIN table2
        ON table1.column_name1=table2.column_name2;
```


In [ ]:
%%sql 
SELECT a.*, t.* 
  FROM Assignments a, Tasks t 
  WHERE a.task_id=t.task_id 
    AND employee_id=2 
    AND status=0
  ORDER BY t.deadline;

## More advanced joins

Suppose we're gradually enrolling our employees in a pension plan.  We create a new table to store their pension plan choices and details.  Now we want to make a little reporting view that shows employees together with their pension choice -- in other words we want to "join" the Pension_Choices table to the Employees table, by lining up along employee_id.

Great!  We just learned how to do that, right?  Not quite -- if an employee hasn't set up their pension yet, we don't want to drop them from the table, we just want to show NULL as the plan_name.

This sort of situation -- where we want to allow the record in the *right hand* table to be not found -- is the reason for SQL's `LEFT OUTER JOIN`.  This requires explicit JOIN syntax:

In [ ]:
%%sql

SELECT e.employee_id, e.name, p.plan_name, p.funding_amount
      FROM Employees e
      LEFT OUTER JOIN Pension_Choices p
      ON   e.employee_id = p.employee_id;

### Further reading

SQL:
- http://www.w3schools.com/sql/default.asp (and http://www.w3schools.com/sql/sql_quickref.asp)

Relational databases and the normal forms:
- http://en.wikipedia.org/wiki/Database_normalization#Normal_forms

## Modifying tables

There are also queries of the form

        DELETE FROM table_name [WHERE condition]
        
        UPDATE table_name SET column_name1=value [WHERE condition];

that delete / modify rows as you might expect.  You can combine both of these with inner queries (and in Postgres, you can combine UPDATE with JOINs) for interesting effects.

To modify columns there are commands of the form

        ALTER TABLE table_name DROP COLUMN column_name;        
        ALTER TABLE table_name ADD COLUMN column_name column_type;
        
for instance

        ALTER TABLE Employees DROP COLUMN phone_number;
        ALTER TABLE Employees ADD COLUMN salary INT;

## SQLite

## Postgresql

### Quick facts

**Running a server:**
- On your DO droplets, once you install postgres it will automatically set up the server to run in the background on startup.  The actual database files are hidden somewhere inside of `/var`.

- Alternatively, you can run a "local" postgres server whose data lies in a specific directory and on a specific port:

>         initdb dir/
>         postgres [-p 8765] -D dir/

(You may want to specify a custom port that is not my made up 8765 -- the default is __5432__.)

**Quick start guide:**
After the above is done and you're running a postgres server locally, just do

         createdb [-p 8765] my_db_name

to create a new database.  When you're done with it

        dropdb --if-exists [-p 8765] my_db_name

To get a command-line console for running commands against your database, just run

        psql [-p 8765] my_db_name

(If it's running on the standard port, you don't need to specify the `-p 8765`.)

By default, this will all use a user named for your Unix username -- for the system Postgres install, this user ("role" in Postgres terms) was created when setting up your droplet.

**Postgres-specific basics:**
To list all tables or find out info about a table do:

        \d
        \d table_name

**Importing (CSV) data:**
If your data is pretty clean, the `\COPY` special command should do it

        \COPY table_name FROM 'path/to/my/file.csv' WITH CSV DELIMITER ',' NULL AS '';
        
**Exporting (CSV) data:** You can use `\COPY` in both directions
        
        \COPY table_name TO 'path/to/my/file.csv' WITH DELIMITER ',' CSV HEADER;

## SQL integration in Python

The most common usage mode for you will be importing / exporting to CSV, or working interactively through `sqlite3` / `psql`.

You can also interact between Python and SQL.  We've been doing it throughout this whole notebook using the `%sql` magic.  There are also libraries for doing it less magically

 - `psycopg2` for communicating with Postgres
 - `sqlalchemy` for communicating with most anything, and some fancy ORM tricks

There's nothing too difficult about either, so we'll just learn them by example in the future.

### What to use?
-------------

**SQLite:**
-------------
_In short:_ Use this if your data is smallish (several GB max) and you want to do some SQL-based processing, _and_ if you know ahead of time that you will not need to do operations in parallel. Probably the preferred choice for mini-projects.

**Pros**
 - It operates on a single flat file on disk, which you can treat like any other file -- copy, scp, rename, etc.
 - It is a single small executable, comes pre-installed on many things, and doesn't require running a server.
 - Extensible (by C code).  It's open source and is small enough to be easy to modify it / write extension modules in C.
 - It has a quick and permissive (of errors) CSV loader, and doesn't stress correctness of schemas.

**Cons**
 - It lacks some useful "advanced" features (in particular, JOINs in UPDATEs have to be faked via subqueries and/or temporary tables).  The workarounds are generally slower than the "right" approach would be.
 - It's not multi-user.  If you're running a long-running computation, you can't do anything else with it.

**Postgresql:**
----------------
_In short:_  The premier free "real" RDBMS (i.e. SQL-type database).  Use it if you need SQL semantics and have more than several GB of data or need multi-user / multi-process access.  
[Exception: If you have lots of data and do lots of easily parallelizable tasks, and will be running either on a cluster on a machine that fits your data in RAM, you may be better off with a specialized SQL-like system: e.g. Hive, Shark, MemSQL (commercial).]

**Pros**
 - Excellent support of SQL features, good performance and `"EXPLAIN ..."` for seeing query plans and tweaking performance.
 - Several nice SQL extensions: JSON-column support, and `Postgis` for GIS-based geographical data types and queries.
 - Extensible (in a variety of languages, _including Python_).
 
**Cons**
 - You have to run a server.
 - The CSV loader is quick on correct data, but simply doesn't except malformed data.  Does stress correctness of schemas.
 
 
 **Other options:**
 ------------------
 - *If your data is small enough to fit _comfortably_ into memory:* 
 
   You can get by with something like `pandas` which allows for much quicker visualization and has better built in statistical support.  (There is also support for running SQL queries against pandas.  It uses sqlite behind the scenes -- beware that doing this has a performance cost over just using sqlite originally.)
 - *If you have have *lots* of data (i.e. more than any one machine you have can store on disk) or if you want to perform a task that calls for cross-machine parallelization (e.g. it makes sense to run on Hadoop):*
 
   There are specialized SQL-like databases made for this, notably `Hive` which is built on top of Hadoop.  (Also: `Shark` and `Impala` which do the same thing with different size and performance characteristics.)
 - *If your data set is large, but you happen to have a _very_ beefy sever (or a cluster of them) with enough RAM to fit it all:*
 
  There are SQL-like databases optimized for in-memory use, including `MemSQL` and also modes of `Shark` and `Impala` mentioned above.

### Exercises
Exercise 2 may be too much of a scraping, rather than SQL, problem.

**Exercise:**

1. Create a SQL database for storing [CBSAs](http://www.census.gov/geo/reference/gtc/gtc_cbsa.html) ([read more](http://en.wikipedia.org/wiki/Core_Based_Statistical_Area)) together with the following basic info: 
    - population (2012); 
    - area; 
    - type;
    - states;

  Clearly describe how many tables you will use, what the keys are, etc.

2. Write a script to populate your database by using some combination of [the OMB deliniation files](http://www.census.gov/population/metro/data/def.html) and [Wikipedia's list of CBSAs](http://en.wikipedia.org/wiki/List_of_Core_Based_Statistical_Areas) and the links therein.  If those links don't work, use `small_data/List1.xls` and `List_of_Core_Based_Statistical_Areas`.

  *Hint:* the OMB files are in Excel.  You can use Python's `xlrd` library to read the data in and `csv` to write out the data to a csv.  Wikipedia's data is an HTML table.  Try using `pandas.read_html` function to save the data as a csv.  You can then load the data in Sqlite.  What's the common key?  You might need to massage the data in Python if the keys do not align.

3. Perform a few joins in SQL.  Give the total urban population for 2010 and 2012 by state.  Get the total central vs. outlying population (see the OMB datafile for the definition).

### Exit Tickets
1. You want to find all comments made by a user by joining the users and comments tables. What kind of join should you use?
1. What are the benefits/drawbacks of doing data analysis in SQL versus Python?
1. Is a primary key necessary to perform a join?

### Spoiler space...

### ...

### ...

### ...

### ...

Here's the sort of answer I'd get for #1:
```SQL
CREATE TABLE CBSA
    ( cbsa_id INTEGER NOT NULL
    , cbsa_name TEXT NOT NULL
    , cbsa_type VARCHAR(4) NOT NULL
    , population INTEGER NOT NULL
    , area INTEGER NOT NULL
    , PRIMARY KEY cbsa_id
    );
CREATE TABLE CBSA_States
    ( cbsa_id INTEGER NOT NULL
    , state_code  CHAR(2) NOT NULL
    , PRIMARY KEY (cbsa_id, state)
    );
```

While for #2, we can first introduce a new helpful tool.  The data came to us in XLS form, so how do we load Excel files into Python?  The easiest way -- if you have Excel installed and don't care about automating the process -- is to export the XLS to CSV, and load the CSV.

An alternate way is to use a library called __xlrd__:

In [ ]:
# We're going to have to read an Excel file!

import xlrd
import urllib2

wb = xlrd.open_workbook(file_contents = urllib2.urlopen("http://www.census.gov/population/metro/files/lists/2013/List1.xls").read())

In [ ]:
wb.sheet_names()

In [ ]:
sheet1 = wb.sheet_by_index(0)

In [ ]:
for i in range(10):
    print sheet1.row(i) 
    print
    

**Remark:** Does this file look like it would make for a well-designed ("normalized") table?

In [ ]:
rows = [sheet1.row(i) for i in range(3, sheet1.nrows-3)]
data = [(r[0].value, r[4].value, r[8].value, r[3].value) for r in rows]
print data[0], data[-1]

In [ ]:
long_to_short_type = {
    'Metropolitan Statistical Area' : 'MSA',
    'Micropolitan Statistical Area' : 'mSA'
}
def reverse_dict(dict):
    return {v:k for k, v in dict.items()}
short_to_long_type = reverse_dict(long_to_short_type)

In [ ]:
import us
long_to_short_state = us.states.mapping('name', 'abbr')

In [ ]:
from sets import Set
cbsa_by_id = {}
cbsa_id_by_name = {}
# Build the ids/names/types/states from the OMB data.
for x in data:
    (cbsa_id_str, type_long, state_long, cbsa_name) = x
    cbsa_id = int(cbsa_id_str)
    if cbsa_id not in cbsa_by_id:
        cbsa_by_id[cbsa_id] = { 
                'cbsa_id': cbsa_id, 
               'cbsa_name': cbsa_name, 
               'cbsa_type': long_to_short_type[type_long], 
               'population' : None,
               'area' : None,
               'states' :  Set([])
        }
        cbsa_id_by_name[cbsa_name] = cbsa_id
    cbsa_by_id[cbsa_id]['states'].add( long_to_short_state[state_long]  )

In [ ]:
cbsa_by_id.values()[0]

In [ ]:
def guess_url(cbsa):
    url_base="http://en.wikipedia.org/wiki/"
    url_rest = (cbsa['cbsa_name']+'_'+short_to_long_type[cbsa['cbsa_type']]).replace(' ', '_')
    return  url_base + url_rest

In [ ]:
import re
from bs4 import BeautifulSoup
def add_info(cbsa):
    url=guess_url(cbsa)
    print url
    soup = BeautifulSoup(urllib2.urlopen(url))
    # Exercise: Fill this in!
    #
    # Perhaps look for any tag that has text, and that this text is (after suitable stripping) = "Population" (resp., "Area")
    # and then search for the next tag which says "Total", and finally take the next td!
    # 
    # This might be a good place to talk about regular expressions...
    #
    return cbsa

In [ ]:
# load wiki table using 

import pandas as pd
tables = pd.read_html("http://en.wikipedia.org/wiki/List_of_Core_Based_Statistical_Areas", header=0)
cbsa_table = tables[0]
# cbsa_table.to_csv("wiki_table.csv")  # saves table as csv file

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*